In [1]:
import pandas as pd
import requests
import json
import ast
from bs4 import BeautifulSoup

In [2]:
restaurant_data_df = pd.read_excel("rw_resturants_2025-07_v3.xlsx")
restaurant_data_df

,borough,neighborhood,primaryCategory,primaryLocation,ctaButtons,tags,shortTitle,slug,image,summary,website,menuFileUrl,collections,mealTypes,restaurantInclusionWeek,ecommerce,rest_week_site_url,image_url
0,Manhattan,Lower East Side,Restaurants,Lower East Side,"{'primary': {'label': {'presale': False, 'gene...","['Korean', 'Lower East Side']",Sae Ron,sae-ron,{'url': '//images.ctfassets.net/1aemqu6a6t65/1...,Sae Ron is a modern Korean kitchen dedicated t...,https://www.saeronnyc.com,NaN,[],['$45 Dinner Price'],"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/sae...,https://images.ctfassets.net/1aemqu6a6t65/1G7R...
1,Manhattan,Times Square/Theatre District,Restaurants,Times Square/Theatre District,"{'primary': {'label': {'presale': False, 'gene...","['Cuban', 'Times Square/Theatre District']",Victor's Cafe,victors-cafe,{'url': '//images.ctfassets.net/1aemqu6a6t65/2...,"Victor’s, in the Theatre District, has been a ...",https://victorscafe.com,NaN,[],"['$45 Dinner Price', '$30 Sunday Lunch/Brunch ...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/vic...,https://images.ctfassets.net/1aemqu6a6t65/2DMb...
2,Manhattan,Times Square/Theatre District,Restaurants,Times Square/Theatre District,"{'primary': {'label': {'presale': False, 'gene...","['Italian', 'Times Square/Theatre District']",AperiBar,aperibar,{'url': '//images.ctfassets.net/1aemqu6a6t65/1...,This chic restaurant and cocktail bar serves u...,https://www.aperibar.com,NaN,[],['$45 Dinner Price'],"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/ape...,https://images.ctfassets.net/1aemqu6a6t65/1TiS...
3,Manhattan,Murray Hill,Restaurants,Murray Hill,"{'primary': {'label': {'presale': False, 'gene...","['Mediterranean', 'Murray Hill']",Leonetta,leonetta,{'url': '//images.ctfassets.net/1aemqu6a6t65/4...,Bringing Mediterranean warmth to Murray Hill a...,https://www.leonettanyc.com,NaN,[],"['$30 Lunch Price', '$30 Sunday Lunch/Brunch P...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/leo...,https://images.ctfassets.net/1aemqu6a6t65/4eyN...
4,Queens,Forest Hills,Restaurants,Forest Hills,"{'primary': {'label': {'presale': False, 'gene...","['Italian', 'Forest Hills']",Tuscan Hills,tuscan-hills,{'url': '//images.ctfassets.net/1aemqu6a6t65/2...,"Tuscan dishes such as the house specialty, cac...",https://www.tuscanhillsnyc.com,NaN,[],"['$30 Lunch Price', '$45 Dinner Price', '$30 S...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/tus...,https://images.ctfassets.net/1aemqu6a6t65/24Hz...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,Manhattan,Harlem,Nightlife,Harlem,"{'primary': {'label': {'presale': False, 'gene...","['French', 'Harlem']",Barawine,barawine-harlem,"{'url': '', 'alt': ''}","This chic wine bar—overseen by Fabrice Warin, ...",https://www.barawine.com,NaN,[],"['$45 Lunch Price', '$45 Dinner Price', '$30 S...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/bar...,NaN
614,Manhattan,Chelsea,Restaurants,Chelsea,"{'primary': {'label': {'presale': False, 'gene...","['Thai', 'Chelsea']",Wan Waan,wan-waan,"{'url': '', 'alt': ''}","The team at Wan Waan, a Thai restaurant in the...",https://wannyc.com,NaN,[],"['$30 Lunch Price', '$45 Dinner Price', '$45 S...","['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",NaN,https://www.nyctourism.com/restaurant-week/wan...,NaN
615,Manhattan,Upper East Side,Restaurants,Upper East Side,"{'primary': {'label': {'presale': False, 'gene...","['Greek', 'Upper East Side']",Yefsi Estiatorio,yefsi-estiatorio,"{'url': '', 'alt': ''}","Yefsi Estiatorio, a Greek restaurant on Ma

In [3]:
def get_page_data(rest_week_site_url):
    page_data = {}
    req = requests.get(rest_week_site_url)
    if req.status_code != 200:
        return {"location_coords": {"lat": "", "lon": ""}, "pdf_url": "", "venueAddress": ""}
        
    soup = BeautifulSoup(req.text)
    next_data = soup.select("#__NEXT_DATA__")[0]
    next_data_json = json.loads(next_data.text)
    fields = next_data_json["props"]["pageProps"]["pageFields"]["fields"]
    page_data["location_coords"] = fields["location"]
    page_data["pdf_url"] = next_data_json["props"]["pageProps"]["page"].get("fileUrl", "")
    page_data["venueAddress"] = fields["venueAddress"]

    return page_data

In [4]:
def get_all_rest_data(search_data):
    page_data = get_page_data(search_data["rest_week_site_url"])
    
    rest_data = {}
    rest_data["name"] = search_data["shortTitle"]
    rest_data["pdf_url"] = page_data["pdf_url"]
    rest_data["slug"] = search_data["slug"]
    rest_data["url"] = search_data["rest_week_site_url"]
    rest_data["venueAddress"] = page_data["venueAddress"]
    rest_data["latitude"] = page_data["location_coords"]["lat"]
    rest_data["longitude"] = page_data["location_coords"]["lon"]
    rest_data["summary"] = search_data["summary"]
    rest_data["website"] = search_data["website"]
    rest_data["collections"] = search_data["collections"]
    rest_data["borough"] = search_data["borough"]
    rest_data["neighborhood"] = search_data["neighborhood"]
    rest_data["primaryCategory"] = search_data["primaryCategory"]
    rest_data["primaryLocation"] = search_data["primaryLocation"]
    rest_data["restaurantInclusionWeek"] = search_data["restaurantInclusionWeek"]
    rest_data["image_url"] = search_data["image_url"]
    
    mealTypes = ast.literal_eval(search_data["mealTypes"])
    rest_data["meal_type"] = "; ".join(mealTypes).replace(" Price", "")

    rest_tags = ast.literal_eval(search_data["tags"].replace("None", '""'))
    rest_data["tags"] = "; ".join(rest_tags).strip("; ")

    return rest_data

In [5]:
%%time
count = 1
total = len(restaurant_data_df)
all_rest_data = []
for index, restaurant in restaurant_data_df.iterrows():
    print(f"{count} out of {total}: {restaurant['shortTitle']}")
    rest_data = get_all_rest_data(restaurant)
    all_rest_data.append(rest_data)
    count += 1

1 out of 618: Sae Ron
2 out of 618: Victor's Cafe
3 out of 618: AperiBar
4 out of 618: Leonetta
5 out of 618: Tuscan Hills
6 out of 618: La Sirene
7 out of 618: Eléa
8 out of 618: Dear Strangers
9 out of 618: Lorenzo's Restaurant, Bar & Cabaret
10 out of 618: Jacob's Pickles
11 out of 618: Yakiniku Futago
12 out of 618: The Tuck Room
13 out of 618: Fushimi - Times Square
14 out of 618: Muteki Udon
15 out of 618: Koloman
16 out of 618: Nar Restaurant
17 out of 618: Loring Place
18 out of 618: Il Buco Alimentari & Vineria
19 out of 618: Amali
20 out of 618: Prohibition
21 out of 618: Taste of India II
22 out of 618: Wallsé
23 out of 618: Torch & Crown Brewing Company
24 out of 618: Iris
25 out of 618: Philippe by Philippe Chow Downtown
26 out of 618: Tasca NYC
27 out of 618: Stella 34 Trattoria
28 out of 618: Tha Phraya
29 out of 618: Balade
30 out of 618: Chili
31 out of 618: La Baraka
32 out of 618: Dos Caminos Meatpacking District
33 out of 618: Ootoya - Union Square
34 out of 618: He

260 out of 618: Ilili Restaurant
261 out of 618: Lele's Roman
262 out of 618: A La Turka
263 out of 618: Rosa Mexicano - Union Square
264 out of 618: Mezze on the River
265 out of 618: Jade Eatery & Lounge
266 out of 618: La Mercerie
267 out of 618: Tarallucci e Vino - NoMad
268 out of 618: Michael's
269 out of 618: Sofrito on the Hudson
270 out of 618: The Ribbon—Upper West Side
271 out of 618: Souvlaki GR - Midtown East
272 out of 618: Fish Cheeks
273 out of 618: Treadwell Park - Upper East Side
274 out of 618: Sant Ambroeus—Brookfield
275 out of 618: Miss Lily's 7A
276 out of 618: The Parlour Room
277 out of 618: Brooklyn Bowl
278 out of 618: Ophelia Lounge NYC
279 out of 618: Shan
280 out of 618: The Terrace and Outdoor Gardens
281 out of 618: Robert
282 out of 618: Docks Oyster Bar
283 out of 618: 5 Napkin Burger - Hell's Kitchen
284 out of 618: Rosa Mexicano - Second Avenue
285 out of 618: American Cut Tribeca
286 out of 618: Lore
287 out of 618: Haven Rooftop
288 out of 618: Cal

511 out of 618: Nangman BBQ
512 out of 618: La Pulperia
513 out of 618: Serra
514 out of 618: Hawksmoor
515 out of 618: Wonderland Bar
516 out of 618: Meadowsweet
517 out of 618: The Russian Tea Room
518 out of 618: Anassa Taverna - Manhattan
519 out of 618: Crave Fishbar - Midtown
520 out of 618: The Dutch
521 out of 618: Le Gratin
522 out of 618: Korali Estiatorio
523 out of 618: Raval Steakhouse
524 out of 618: Charoen Krung Thai Restaurant
525 out of 618: Sojourn Social
526 out of 618: Zoì Mediterranean Cuisine
527 out of 618: Paris Bar
528 out of 618: Little Alley
529 out of 618: Zaytinya
530 out of 618: Amaranth
531 out of 618: Anassa Taverna - Downtown
532 out of 618: Nizuc
533 out of 618: 8282
534 out of 618: Safari
535 out of 618: La Pecora Bianca Bryant Park
536 out of 618: Carib-Soul Bk
537 out of 618: Atlantis Prime
538 out of 618: SEA by Jungsik
539 out of 618: Flatiron Steakhouse
540 out of 618: Oyamel
541 out of 618: Murano Ristorante
542 out of 618: Mamazul
543 out of 6

In [6]:
all_rest_data_df = pd.DataFrame(all_rest_data)
all_rest_data_df

,name,pdf_url,slug,url,venueAddress,latitude,longitude,summary,website,collections,borough,neighborhood,primaryCategory,primaryLocation,restaurantInclusionWeek,image_url,meal_type,tags
0,Sae Ron,https://nyc-tourism-public.s3.amazonaws.com/sr...,sae-ron,https://www.nyctourism.com/restaurant-week/sae...,"99 Stanton St.,Manhattan,10002,NY",40.721162,-73.988116,Sae Ron is a modern Korean kitchen dedicated t...,https://www.saeronnyc.com,[],Manhattan,Lower East Side,Restaurants,Lower East Side,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/1G7R...,$45 Dinner,Korean; Lower East Side
1,Victor's Cafe,https://nyc-tourism-public.s3.amazonaws.com/sr...,victors-cafe,https://www.nyctourism.com/restaurant-week/vic...,"236 W. 52nd St.,Manhattan,10019,NY",40.762891,-73.984119,"Victor’s, in the Theatre District, has been a ...",https://victorscafe.com,[],Manhattan,Times Square/Theatre District,Restaurants,Times Square/Theatre District,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/2DMb...,$45 Dinner; $30 Sunday Lunch/Brunch; $45 Sunda...,Cuban; Times Square/Theatre District
2,AperiBar,https://nyc-tourism-public.s3.amazonaws.com/sr...,aperibar,https://www.nyctourism.com/restaurant-week/ape...,"Luma Hotel Times Square, 120 W. 41st St.,Manha...",40.754376,-73.985804,This chic restaurant and cocktail bar serves u...,https://www.aperibar.com,[],Manhattan,Times Square/Theatre District,Restaurants,Times Square/Theatre District,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/1TiS...,$45 Dinner,Italian; Times Square/Theatre District
3,Leonetta,,leonetta,https://www.nyctourism.com/restaurant-week/leo...,"181 Lexington Ave.,Manhattan,10016,NY",40.744577,-73.980774,Bringing Mediterranean warmth to Murray Hill a...,https://www.leonettanyc.com,[],Manhattan,Murray Hill,Restaurants,Murray Hill,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/4eyN...,$30 Lunch; $30 Sunday Lunch/Brunch,Mediterranean; Murray Hill
4,Tuscan Hills,,tuscan-hills,https://www.nyctourism.com/restaurant-week/tus...,"110-60 Queens Blvd.,Queens,11375,NY",40.718972,-73.839351,"Tuscan dishes such as the house specialty, cac...",https://www.tuscanhillsnyc.com,[],Queens,Forest Hills,Restaurants,Forest Hills,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/24Hz...,$30 Lunch; $45 Dinner; $30 Sunday Lunch/Brunch...,Italian; Forest Hills
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,Barawine,https://nyc-tourism-public.s3.amazonaws.com/sr...,barawine-harlem,https://www.nyctourism.com/restaurant-week/bar...,"200 Lenox Ave.,Manhattan,10027,NY",40.804589,-73.947483,"This chic wine bar—overseen by Fabrice Warin, ...",https://www.barawine.com,[],Manhattan,Harlem,Nightlife,Harlem,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,$45 Lunch; $45 Dinner; $30 Sunday Lunch/Brunch...,French; Harlem
614,Wan Waan,,wan-waan,https://www.nyctourism.com/restaurant-week/wan...,"200 Eighth Ave.,Manhattan,10011,NY",40.743481,-73.999503,"The team at Wan Waan, a Thai restaurant in the...",https://wannyc.com,[],Manhattan,Chelsea,Restaurants,Chelsea,"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",NaN,$30 Lunch; $45 Dinner; $45 Sunday Lunch/Brunch...,Thai; Chelsea
615,Yefsi Estiatorio,https://nyc-tourism-public.s3.amazonaws.com/sr...,yefsi-estiatorio,https://www.nyctourism.com/restaurant-week/yef...,"1481 York Ave.,Manhattan,10075,NY",40.771046,-73.951026,"Yefsi Estiatorio, a Greek restaurant on Manhat...",http://www.yefsiestiatorio.com,[],Manhattan,Upper East Side,Restaurants,Upper East Side,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,$30 Lunch; $45 Dinner,Greek; Upper East Side
616,Amélie - Upper West Side,,amelie-upper-west-side,https://www.nyctourism.com/restaurant-week/ame...,"566 Amsterdam Ave.,Manhattan,10024,NY",40.788751,-73.974624,""

In [7]:
all_rest_data_df.to_excel("rw_resturants_2025-07_full_v2.xlsx", index=False)